In [2]:
#!/usr/bin/env python

# It seems that text preprocessing is the first step for many deep learning work
## So, I'd better wrire down those basic methods I foten used

# allow foreign languages in the text
import sys  
stdout = sys.stdout
reload(sys)  
sys.setdefaultencoding('utf8')
sys.stdout = stdout

import nltk
from nltk.stem.porter import *

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('maxent_treebank_pos_tagger')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

## MORE RESOURCES HERE:
# For more, check my code files here: https://github.com/hanhanwu/Hanhan_Play_With_Social_Media
# Sapcy modules for download: https://spacy.io/usage/models
# Spacy tutorial: https://www.analyticsvidhya.com/blog/2017/04/natural-language-processing-made-easy-using-spacy-%E2%80%8Bin-python/
# Textacy for preproecssing, feature extraction, text summarization with LDA, NMF, LSA:
## https://github.com/hanhanwu/Hanhan_NLP/blob/master/textacy_explore/go_through_features.py

In [4]:
stemmer = PorterStemmer()
stopwords = nltk.corpus.stopwords.words('english')

In [5]:
s1 = """My life is so busy when it's close to the end of the year. 
        Most of the time I have to work day and night just in order to have my own time on weekends.
        Then during the weekends, I need to sleep more, and do more house work, go shopping, 
        sometimes hag out with girlfirends. Sometimes just want to read a book, and I need to do my side projects
        all the time. Sometimes, even during the weekend, had to work since I want to be outstanding.
        I want to learn broader and deeper in cyber security, and I'm taking actions, I'm using my little time
        to learn penetration testing, to learn from hackers in DEF CON. I want to be excellent and always have
        my passion in data science in cyber secuirty. There are many other things to deal with, some are really annoying.
        They keep remind you, life can never be perfect. I still love Emmanuel, miss him and sometimes wonder what's
        happening there, but my busy life does not allow me to think too much."""

In [6]:
# extract sentences from text, and lower all the case
sentences = [s.lower() for s in nltk.tokenize.sent_tokenize(s1)]
print sentences[7]
print sentences[9]

there are many other things to deal with, some are really annoying.
i still love emmanuel, miss him and sometimes wonder what's
        happening there, but my busy life does not allow me to think too much.


In [8]:
# remove punctuation
# With this method, can't convter to cant;
# With other nltk method, can't convert to can, t 2 tokens
import string

sentences = [s.translate(None, string.punctuation) for s in sentences]
print sentences

['my life is so busy when its close to the end of the year', 'most of the time i have to work day and night just in order to have my own time on weekends', 'then during the weekends i need to sleep more and do more house work go shopping \n        sometimes hag out with girlfirends', 'sometimes just want to read a book and i need to do my side projects\n        all the time', 'sometimes even during the weekend had to work since i want to be outstanding', 'i want to learn broader and deeper in cyber security and im taking actions im using my little time\n        to learn penetration testing to learn from hackers in def con', 'i want to be excellent and always have\n        my passion in data science in cyber secuirty', 'there are many other things to deal with some are really annoying', 'they keep remind you life can never be perfect', 'i still love emmanuel miss him and sometimes wonder whats\n        happening there but my busy life does not allow me to think too much']


In [9]:
# extract tokens (non-stopwords) from sentences
tokens = [w for s in sentences for w in s.split() if w not in stopwords and len(w)<20]
print tokens[7]
print tokens[9]

day
order


In [10]:
# calculate words similarity
import Levenshtein

print Levenshtein.distance(tokens[7], tokens[9])

4


In [11]:
# Stemming
tokens9 = [w for w in sentences[9].split() if w not in stopwords and len(w)<20]
print tokens9
stemmed_words = [stemmer.stem(tokens9[i]) for i in range(len(tokens9))]
print stemmed_words

['still', 'love', 'emmanuel', 'miss', 'sometimes', 'wonder', 'whats', 'happening', 'busy', 'life', 'allow', 'think', 'much']
['still', 'love', 'emmanuel', u'miss', u'sometim', 'wonder', u'what', u'happen', u'busi', 'life', 'allow', 'think', 'much']


In [12]:
# Get entities, here it's just NN entities

def get_NN_entities(query):
    sentences = nltk.tokenize.sent_tokenize(query)
    token_sets = [nltk.tokenize.word_tokenize(s) for s in sentences]
    pos_tagged_token_sets = [nltk.pos_tag(t) for t in token_sets]
    pos_tagged_tokens = [t for v in pos_tagged_token_sets for t in v]
    
    all_entities = []
    previous_pos = None
    current_entities = []
    for (entity, pos) in pos_tagged_tokens:
        if previous_pos == pos and pos.startswith('NN'):  # here you define entities you want
            current_entities.append(entity.lower())
        elif pos.startswith('NN'):  # here you define entities you want
            if current_entities != []:
                all_entities.append(' '.join(current_entities))
            current_entities = [entity.lower()]
        previous_pos = pos
    return list(set([entity for entity in all_entities]))

get_NN_entities(s1)

['things',
 'hackers',
 'emmanuel',
 'actions',
 'year',
 'girlfirends',
 'weekend',
 'weekends',
 'book',
 'passion',
 'def con',
 'life',
 'shopping',
 'cyber secuirty',
 "what's",
 'night',
 'penetration',
 'house work',
 'end',
 'data',
 'day',
 'projects',
 'science',
 'side',
 'remind',
 'time',
 'security',
 'order']

In [53]:
# Get NN entities with Spacy
doc = nlp(s1.decode('utf8'))
for np in doc.noun_chunks:
    print np.text, np.root.dep_, np.root.head.text

My life nsubj is
it nsubj 's
the end pobj to
the year pobj of
the time pobj of
I nsubj have
order pobj in
my own time dobj have
weekends pobj on
the weekends pobj during
I nsubj need
more house work dobj do
shopping dobj go
girlfirends pobj with
a book dobj read
I nsubj need
my side projects dobj do
the weekend pobj during
I nsubj want
I nsubj want
cyber security pobj in
I nsubj taking
actions dobj taking
I nsubj using
my little time dobj using
penetration testing dobj learn
hackers pobj from
DEF CON pobj in
I nsubj want
my passion dobj have
data science pobj in
cyber secuirty pobj in
many other things attr are
They nsubj keep
you dobj remind
life nsubj be
I nsubj love
Emmanuel dobj love
him dobj miss
what nsubj 's
my busy life nsubj allow
me nsubj think


In [26]:
# Lemmatisation - NLTK method
from nltk.stem import WordNetLemmatizer

wordnet_lemmatizer = WordNetLemmatizer()

tokens9 = [w for w in sentences[9].split() if w not in stopwords and len(w)<20]
print tokens9
lemmatised_words = [wordnet_lemmatizer.lemmatize(tokens9[i]) for i in range(len(tokens9))]
print lemmatised_words

['still', 'love', 'emmanuel', 'miss', 'sometimes', 'wonder', 'whats', 'happening', 'busy', 'life', 'allow', 'think', 'much']
['still', 'love', 'emmanuel', 'miss', 'sometimes', 'wonder', 'whats', 'happening', 'busy', 'life', 'allow', 'think', 'much']


In [34]:
# Lemmatisation - spacy method
# type `python -m spacy download en` in your terminal to download spacy english
import spacy

nlp=spacy.load("en")

for token in nlp(sentences[9].decode('utf8')):
    print(token,token.lemma_)

(i, u'i')
(still, u'still')
(love, u'love')
(emmanuel, u'emmanuel')
(miss, u'miss')
(him, u'-PRON-')
(and, u'and')
(sometimes, u'sometimes')
(wonder, u'wonder')
(what, u'what')
(s, u's')
(
        , u'\n        ')
(happening, u'happen')
(there, u'there')
(but, u'but')
(my, u'-PRON-')
(busy, u'busy')
(life, u'life')
(does, u'do')
(not, u'not')
(allow, u'allow')
(me, u'-PRON-')
(to, u'to')
(think, u'think')
(too, u'too')
(much, u'much')


In [ ]:
# word embeding - word2vector

## Spacy
# type `python -m spacy download en_core_web_lg`
nlp = spacy.load('en_vectors_web_lg')

## gensim
#!pip install gensim
# download Google pre-trained word vectors: https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit
from gensim.models.keyedvectors import KeyedVectors
word_vectors=KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin',binary=True)
print word_vectors[tokens9[7]]
tokens9 = [w for w in sentences[9].split() if w not in stopwords and len(w)<20]
tokens7 = [w for w in sentences[7].split() if w not in stopwords and len(w)<20]
sentence=[tokens7, tokens9]
model = gensim.models.Word2Vec(sentence, min_count=1,size=300,workers=4)

## Spark word2vector
# https://spark.apache.org/docs/2.2.0/ml-features.html#word2vec

In [38]:
# part-of_speech tagging

for token in nlp(sentences[7].decode('utf8')):
    print(token,token.pos_)
    
for token in nlp(sentences[9].decode('utf8')):
    print(token,token.pos_)

(there, u'ADV')
(are, u'VERB')
(many, u'ADJ')
(other, u'ADJ')
(things, u'NOUN')
(to, u'PART')
(deal, u'VERB')
(with, u'ADP')
(some, u'DET')
(are, u'VERB')
(really, u'ADV')
(annoying, u'ADJ')
(i, u'PRON')
(still, u'ADV')
(love, u'VERB')
(emmanuel, u'NOUN')
(miss, u'VERB')
(him, u'PRON')
(and, u'CCONJ')
(sometimes, u'ADV')
(wonder, u'VERB')
(what, u'NOUN')
(s, u'VERB')
(
        , u'SPACE')
(happening, u'VERB')
(there, u'ADV')
(but, u'CCONJ')
(my, u'ADJ')
(busy, u'ADJ')
(life, u'NOUN')
(does, u'VERB')
(not, u'ADV')
(allow, u'VERB')
(me, u'PRON')
(to, u'PART')
(think, u'VERB')
(too, u'ADV')
(much, u'ADV')


In [44]:
# Named Entity Reognition

doc = nlp(s1.decode('utf8'))
for ent in doc.ents:
    if ent.text.strip() != '' and ent.text is not None:
        print ent.text, ent.label_

the end of the year DATE
the weekend DATE
DEF CON ORG
Emmanuel PERSON


In [52]:
# text summarization
from gensim.summarization import summarize

s2 = """It's raining whole day today. Fortunately, I ordered all the furnitures yesterday when it was not raining. 
     And also lucky, that yesterday, I watched Star Wars when before it became a heavy rain. So many people went to
     the theater just for Star Wars. They used multiple largest room for this movie and you even cannnot see any 
     empty seat there. But to my surprise, after the movie, people have left large amount of trash in the theater. 
     I'm still a big fan of Star Wars, it's just this one did not make me feel excited. Not sure why I just feel the
     original feeling about destiny has become subtle in this movie and the way it captures emotions has also become
     less careful. Jokes added in the movie just like many other Disney movies, but can no longer make me feel it's a
     legend. It's barrative is also predictable and won't make me feel that I want to know what will happen next. Anyway
     people still love Star Wars. My favorite Star Wars are I, II, III, I like both the actors and the stories there.
     This year, near the end of the year, I allowed myself to watch some movie since there are really some great movies
     at the time and working all the time did make me feel stressful. Anyway, guess this year my favorite movie is CoCo,
     especially love that song, Remember Me. 
     What is Baby Emmanuel doing at this moment? I was surprised that till now, the news is still reporting the wild
     fire in LA. It's winter here now and it rains heavily here, but in LA, it's still fighting the huge wild fire...
     Christmas is coming, this year unfortunately, I cannot go back home, I'm feeling sad but keep positive, I will
     have my furnitures delivered, get some really good desert and books, I can enjoy the Christmas by reading the books
     enjoying the desert. I miss Baby Emmauel so much, would he know, I always think it's very cool that he likes playing
     video games?"""

summarize(s2)

u"And also lucky, that yesterday, I watched Star Wars when before it became a heavy rain.\nthe theater just for Star Wars.\nI'm still a big fan of Star Wars, it's just this one did not make me feel excited.\npeople still love Star Wars.\nMy favorite Star Wars are I, II, III, I like both the actors and the stories there."

In [67]:
# Dependency Parsing
doc = nlp(''.join(s2.split('\n')).decode('utf8'))
star_wars_sentences = [sent for sent in doc.sents if 'star wars' in sent.string.lower()]
print star_wars_sentences

for sent in star_wars_sentences:
    print "************************"
    for wd in sent:
        print wd, ':', str(list(wd.children))

[And also lucky, that yesterday, I watched Star Wars when before it became a heavy rain., So many people went to     the theater just for Star Wars., I'm still a big fan of Star Wars, Anyway     people still love Star Wars., My favorite Star Wars are I, II, III, I like both the actors and the stories there.     ]
************************
And : [lucky]
also : []
lucky : [also, ,]
, : []
that : []
yesterday : []
, : []
I : []
watched : [And, that, yesterday, ,, I, Wars, became, .]
Star : []
Wars : [Star]
when : []
before : []
it : []
became : [when, before, it, rain]
a : []
heavy : []
rain : [a, heavy]
. : []
************************
So : []
many : [So]
people : [many]
went : [people, to, theater, .]
to : [    ]
     : []
the : []
theater : [the, for]
just : []
for : [just, Wars]
Star : []
Wars : [Star]
. : []
************************
I : []
'm : [I, still, fan]
still : []
a : []
big : []
fan : [a, big, of]
of : [Wars]
Star : []
Wars : [Star]
************************
Anyway : [    ]
    

In [90]:
# sentiment analysis

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

sentences = [('Emmanuel is very adorable!', 'pos'),          
         ('Baby Emmanuel also likes Star Wars and Totoro.', 'pos'),
         ('I love ice-cream', 'pos'),
         ('Christmas is coming!', 'pos'),
         ("Baby Emmanuel will return back to my life.", 'pos'),
         ('Baby Emmauel often hurt himself for no reason.', 'neg'),
         ('Emmanuel does not want to see me again', 'neg'),
         ("I can't go home this year", 'neg'),
         ('Baby Emmanuel is gaining more fat', 'neg'),          
         ('Baby Emmanuel forgot me', 'neg'),
         ('Totoro likes baby Emmanuel', 'pos'),     
         ('Christmas is coming, but I cannot go back home.', 'neg'),
         ("Tomorrow is Monday again.", 'neg'),
         ("Holiday is coming!", 'pos'),
         ('Baby Emmanuel is happy.', 'pos'),
         ("LA has wild fire.", 'neg')]

analyzer = SentimentIntensityAnalyzer()
for sentence in sentences:
    vs = analyzer.polarity_scores(sentence[0])
    print("{:-<65} {}".format(sentence[0], str(vs)), "Hanhan's label:"  + sentence[1])

(u"Emmanuel is very adorable!--------------------------------------- {'neg': 0.0, 'neu': 0.442, 'pos': 0.558, 'compound': 0.5838}", u"Hanhan's label:pos")
(u"Baby Emmanuel also likes Star Wars and Totoro.------------------- {'neg': 0.29, 'neu': 0.484, 'pos': 0.226, 'compound': -0.2023}", u"Hanhan's label:pos")
(u"I love ice-cream------------------------------------------------- {'neg': 0.0, 'neu': 0.192, 'pos': 0.808, 'compound': 0.6369}", u"Hanhan's label:pos")
(u"Christmas is coming!--------------------------------------------- {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}", u"Hanhan's label:pos")
(u"Baby Emmanuel will return back to my life.----------------------- {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}", u"Hanhan's label:pos")
(u"Baby Emmauel often hurt himself for no reason.------------------- {'neg': 0.483, 'neu': 0.517, 'pos': 0.0, 'compound': -0.6808}", u"Hanhan's label:neg")
(u"Emmanuel does not want to see me again--------------------------- {'neg': 0.14